A formulation of Poisson equation involves introducing vector variable, namely negative flux $\boldsymbol{\sigma} = -\nabla u$. The PDEs read
\begin{equation}
    \begin{aligned}
    \boldsymbol{\sigma} + \nabla u &= 0 \quad \text{in } \Omega, \\
    \nabla \cdot \boldsymbol{\sigma} &= f \quad \text{in } \Omega,
    \end{aligned} 
\end{equation}
with boundary conditions
\begin{equation}
    \begin{aligned}
    u &= u_{0} \quad \text{on } \Gamma_{D}, \\
    -\boldsymbol{\sigma}\cdot \mathbf{n} &= g \quad \text{on }\Gamma_{N}.
    \end{aligned}
\end{equation}

After multiplying by test functions $\boldsymbol{\tau}$ and $v$, integrating over the domain, and integrating term $\nabla\cdot \boldsymbol{\sigma} v$ by parts, one obtains the following variationa formulation: find $\boldsymbol{\sigma} \in \Sigma$ and $v \in V$ satisfying

\begin{equation}
\begin{aligned}
\int_{\Omega} (\boldsymbol{\sigma}\cdot \boldsymbol{\tau} + \nabla u \cdot \boldsymbol{\tau}) \mathrm{d} x &= 0 \quad \forall \tau \in \Sigma, \\
\int_{\Omega} \boldsymbol{\sigma}\cdot \nabla v \mathrm{d} x &= -\int_{\Omega} f v \mathrm{d} x - \int_{\Gamma_{N}} g v \mathrm{d} x\quad \forall v \in V
\end{aligned}
\end{equation}

In [1]:
from dolfin import *

In [2]:
mesh = UnitSquareMesh(16, 16, "crossed")

In [15]:
k_order = 2
DRT = FiniteElement("DRT", mesh.ufl_cell(), k_order)
CG = FiniteElement("CG", mesh.ufl_cell(), k_order+1)
W = FunctionSpace(mesh, MixedElement([DRT, CG]))

In [20]:
# Define source function
f = Expression("10*exp(-(pow(x[0] - 0.5, 2) \
               + pow(x[1] - 0.5, 2)) / 0.02)", degree=2)
g = Expression("sin(5.0 * x[0])", degree=2)

In [23]:
# Define the boundary condition
def boundary(x, on_boundary):
    return near(x[0], 0, DOLFIN_EPS_LARGE) \
            or near(x[0], 1.0, DOLFIN_EPS_LARGE)

bc = DirichletBC(W.sub(1), 0.0, boundary)

In [31]:
# Define trial and test function
(sigma, u) = TrialFunctions(W)
(tau, v) = TestFunctions(W)

# Define variational form
L = - (f*v*dx + g*v*ds)
a = (dot(sigma, tau) + dot(grad(u), tau) + dot(sigma, grad(v)))*dx

sol = Function(W)
